In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
%matplotlib notebook
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook
import cv2

# Import image

In [106]:
import glob

start = time.time()

LR = [cv2.imread(file).astype(np.float32) for file in glob.glob("./train_set/LR/*.jpg")]
HR = [cv2.imread(file).astype(np.float32) for file in glob.glob("./train_set/HR/*.jpg")]
end = time.time()
print('Time:',end - start)

Time: 17.45333170890808


###### Metric

In [107]:
import math

def MSE(y_true,y_predict):
    temp = (y_true - y_predict)**2
    return sum(temp.reshape(-1))/(temp.shape[0]*temp.shape[1]*temp.shape[2])

def psnr(y_true,y_predict):
    temp = (y_true - y_predict)**2
    mse = sum(temp.reshape(-1))/(temp.shape[0]*temp.shape[1]*temp.shape[2])
    MAXI = 225
    return 20*math.log10(MAXI)-10*math.log10(mse)

# Use traditional method

In [108]:
LR_original = [cv2.imread(file) for file in glob.glob("./train_set/LR/*.jpg")]

LR_to_HR = [cv2.resize(lr, (lr.shape[1]*2, lr.shape[0]*2), interpolation = cv2.INTER_NEAREST) for lr in LR_original] 

In [109]:
start = time.time()

m = []
for i in np.arange(1500):
    m.append(psnr(HR[i],LR_to_HR[i]))

end = time.time()
print('psnr:',np.mean(m))
print('Time:',end - start)


psnr: 24.696329045027174
Time: 303.26858949661255


# Get Features X and respond y using baseline

### Define basic parameters

In [110]:
seed = 1000
channels = np.arange(3)
sample_size = 100

In [111]:
# Get X and y for single pair of LR and HR
def get_X_and_y(LR,HR,n_sample = sample_size,seed = seed):
    # determine seed
    np.random.seed(seed)
    
    # Find neighbor
    def get_neighbor_X(a,i,j):
        return([a[i-1,j-1],a[i-1,j],a[i-1,j+1],a[i,j-1],a[i,j+1],a[i+1,j-1],a[i+1,j],a[i+1,j+1]],a[i,j])

    def get_neighbor_y(a,i,j):
        return([a[i,j],a[i+1,j],a[i,j+1],a[i+1,j+1]])
    
    # padding LR image
    BLACK = [0, 0, 0]
    image_padding = cv2.copyMakeBorder(LR, 1 , 1, 1, 1, cv2.BORDER_CONSTANT, value=BLACK)
    
    y1 = np.zeros((1*n_sample,4))
    y2 = np.zeros((1*n_sample,4))
    y3 = np.zeros((1*n_sample,4))
    
    X1 = np.zeros((1*n_sample,8))
    X2 = np.zeros((1*n_sample,8))
    X3 = np.zeros((1*n_sample,8))
    
    result = [X1,X2,X3]
    Y = [y1,y2,y3]
    
#     height = LR.shape[1]
#     width = LR.shape[0]
    width = LR.shape[1]
    height = LR.shape[0]
    
    # Random pick n_sample point per image
    pts_row = np.random.randint(1, height + 1,size = n_sample)
    pts_col = np.random.randint(1, width + 1,size = n_sample)
    
    for X,y,channel in zip(result,Y,channels):
        index = 0
        for i,j in zip(pts_row,pts_col):
            X_neighbor,central = get_neighbor_X(image_padding[:,:,channel],i,j)
            y_neigbor = get_neighbor_y(HR[:,:,channel],2*(i-1),2*(j-1))
            # Get X
            X[index] = X_neighbor - central
            # Get y
            y[index] = y_neigbor - central
            index +=1
    
    
    # Stack X&Y to 3d
    return np.dstack(result),np.dstack(Y)

In [112]:
def get_Whole_X_and_y(LR,HR,n_sample = sample_size,seed = seed):
    flag = 0
    for lr,hr in tqdm_notebook(zip(LR,HR)):
        if (flag == 0):
            X,y = get_X_and_y(lr,hr,n_sample,seed)
            flag = 1
        else:
            X_temp,y_temp = get_X_and_y(lr,hr,n_sample,seed)
            X = np.vstack([X,X_temp])
            y = np.vstack([y,y_temp])
    return X,y

In [113]:
# n_sample should be 1000
start = time.time()
X,y = get_Whole_X_and_y(LR,HR,n_sample = sample_size)
end = time.time()
print('Time:',end - start)

Time: 20.721944093704224


In [114]:
print(X.shape,y.shape)

(150000, 8, 3) (150000, 4, 3)


# Train & Test split

In [115]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle = True ,random_state=seed)

In [116]:
print(X_train.shape,y_train.shape)

(120000, 8, 3) (120000, 4, 3)


In [117]:
print(X_test.shape,y_test.shape)

(30000, 8, 3) (30000, 4, 3)


# Prepare functions

###### Prediction (12 models)

In [119]:
# test paramter == True means you want to predict X_test, False mean you want to predict X_train
# model_list = [model1,model2,model3,model4,model5,model6,model7,model8,model9,model10,model2,model3]
def Prediction(X_tr,y_tr,X_ts,model_list,test = True):
    
    if (test == False):
        X_test = X_tr
        
    else:
        X_test = X_ts
    
    prediction1 = np.zeros((X_test.shape[0],1,3))
    prediction2 = np.zeros((X_test.shape[0],1,3))
    prediction3 = np.zeros((X_test.shape[0],1,3))
    prediction4 = np.zeros((X_test.shape[0],1,3))
    
    predictions = [prediction1,prediction2,prediction3,prediction4]
    
    index = 0
    for channel in channels:
        for i,prediction in enumerate(predictions):
            model_list[index].fit(X_tr[:,:,channel],y_tr[:,i,channel].reshape(-1,1))
            prediction[:,:,channel] = model_list[index].predict(X_test[:,:,channel]).reshape(-1,1)
            index += 1
    return np.concatenate(predictions,axis = 1)


###### CV

In [120]:
from sklearn.model_selection import KFold

# Output testing error and training error
def cross_validation(X_train,y_train,model_list,metric,folds= 3):
    kf = KFold(n_splits=folds)
    val_error = []
    tr_error = []
    for train_index, val_index in tqdm_notebook(kf.split(X_train)):
        X_tr,y_tr = X_train[train_index],y_train[train_index]
        X_val,y_val = X_train[val_index],y_train[val_index]
        
        y_predict_val = Prediction(X_tr,y_tr,X_val,model_list,test = True)
        y_predict_tr = Prediction(X_tr,y_tr,X_val,model_list,test = False)
        # Metric 
        val_error.append(round(metric(y_val,y_predict_val),2)) # change metric here
        tr_error.append(round(metric(y_tr,y_predict_tr),2)) # change metric here
        
    return val_error,tr_error
        

###### Grid Search ???

In [121]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

def my_grid_search(X_train,y_train,model,parameters,cv = 3,n_jobs = 6):
    
    
    reg = GridSearchCV(model,
                       cv=cv,
                      param_grid = parameters,
                      n_jobs = n_jobs)
    reg.fit(X_train, y_train)
    
    print('Best params',reg.best_params_)
    print('Best score:',reg.score(X_train, y_train))
    
    return reg.best_params_

def grid_search_all_model(X_train,y_train,model,parameters,cv = 3):
    best_params = []
    index = 0
    for channel in channels:
        for i in np.arange(4):
            X_tr = X_train[:,:,channel]
            y_tr = y_train[:,i,channel]
            
            print('Model',index)
            model = model.fit(X_tr,y_tr)
            best_params.append(my_grid_search(X_tr,y_tr,model,parameters,cv = cv))
            index += 1
            
    return best_params

# Train using base model (gradient boosting)

###### Grid Search

In [45]:
from sklearn.ensemble import GradientBoostingRegressor

parameters_tunning = {
    'learning_rate':[0.01, 0.1, 0.2, 0.3], # tuning (started with higher learning rate), hight value --> overfit
    'min_samples_split' : [2, 5, 10], # tuning,high values --> under-fitting
    'min_samples_leaf' : [2, 4,8,10], # tuning, similar to min_samples_split
}

model = GradientBoostingRegressor()

In [46]:
start = time.time()
best = grid_search_all_model(X_train,y_train,model,parameters_tunning,cv = 3)

end = time.time()
print('Time:',end - start)

Best params {'learning_rate': 0.2, 'min_samples_split': 2, 'min_samples_leaf': 10}
Best score: 0.36883367761185804
Best params {'learning_rate': 0.1, 'min_samples_split': 5, 'min_samples_leaf': 4}
Best score: 0.35356424908904105
Best params {'learning_rate': 0.1, 'min_samples_split': 10, 'min_samples_leaf': 10}
Best score: 0.354804349407137
Best params {'learning_rate': 0.1, 'min_samples_split': 2, 'min_samples_leaf': 10}
Best score: 0.35465534772572105
Best params {'learning_rate': 0.2, 'min_samples_split': 5, 'min_samples_leaf': 10}
Best score: 0.3912244087675837
Best params {'learning_rate': 0.1, 'min_samples_split': 5, 'min_samples_leaf': 10}
Best score: 0.3728838152368016
Best params {'learning_rate': 0.1, 'min_samples_split': 10, 'min_samples_leaf': 10}
Best score: 0.3714308925973806
Best params {'learning_rate': 0.1, 'min_samples_split': 10, 'min_samples_leaf': 8}
Best score: 0.36988781655361
Best params {'learning_rate': 0.2, 'min_samples_split': 5, 'min_samples_leaf': 8}
Best 

In [50]:
best

[{'learning_rate': 0.2, 'min_samples_leaf': 10, 'min_samples_split': 2},
 {'learning_rate': 0.1, 'min_samples_leaf': 4, 'min_samples_split': 5},
 {'learning_rate': 0.1, 'min_samples_leaf': 10, 'min_samples_split': 10},
 {'learning_rate': 0.1, 'min_samples_leaf': 10, 'min_samples_split': 2},
 {'learning_rate': 0.2, 'min_samples_leaf': 10, 'min_samples_split': 5},
 {'learning_rate': 0.1, 'min_samples_leaf': 10, 'min_samples_split': 5},
 {'learning_rate': 0.1, 'min_samples_leaf': 10, 'min_samples_split': 10},
 {'learning_rate': 0.1, 'min_samples_leaf': 8, 'min_samples_split': 10},
 {'learning_rate': 0.2, 'min_samples_leaf': 8, 'min_samples_split': 5},
 {'learning_rate': 0.1, 'min_samples_leaf': 8, 'min_samples_split': 2},
 {'learning_rate': 0.1, 'min_samples_leaf': 10, 'min_samples_split': 2},
 {'learning_rate': 0.1, 'min_samples_leaf': 10, 'min_samples_split': 2}]

In [123]:
start = time.time()

model_list = []

for i in np.arange(12):
    model_list.append(GradientBoostingRegressor(**best[i],random_state = seed))

end = time.time()
print('Time:',end - start)

Time: 0.0002956390380859375


### Do CV

###### n_sample = 100

In [130]:
start = time.time()

val_error,tr_error = cross_validation(X_train,y_train,model_list,psnr,folds= 3)
print(' Validation error:',val_error,'\n','Train error:',tr_error,'\n\n')

end = time.time()
print('Time:',end - start)

 Validation error: [24.95, 25.11, 25.07] 
 Train error: [25.38, 25.3, 25.32] 


Time: 325.673122882843


###### n_sample = 1000

In [94]:
start = time.time()

val_error,tr_error = cross_validation(X_train,y_train,model_list,psnr,folds= 3)
print(' Validation error:',val_error,'\n','Train error:',tr_error,'\n\n')

end = time.time()
print('Time:',end - start)


 Validation error: [24.93, 24.89, 24.9] 
 Train error: [24.93, 24.95, 24.95] 


Time: 4603.56507897377


### Predict

###### n_sample = 100

In [54]:
X_test.shape

(30000, 8, 3)

In [55]:
start = time.time()

prediction = Prediction(X_train,y_train,X_test,model_list,test = True)
print('PSNR:',psnr(prediction, y_test),'\n\n')

end = time.time()
print('Time:',end - start)

PSNR: 25.0512282352978 


Time: 98.03701448440552


###### n_sample = 1000

In [95]:
start = time.time()

prediction = Prediction(X_train,y_train,X_test,model_list,test = True)
print('PSNR:',psnr(prediction, y_test),'\n\n')

end = time.time()
print('Time:',end - start)


PSNR: 24.90888108941163 


Time: 1338.4358065128326


In [127]:
# from xgboost import XGBRegressor    

# start = time.time()

# model_list_xgb = []
# model = XGBRegressor(n_jobs = 6,
#                      eval_metric = 'rmse',
#                      random_state = seed)

# params_xgb = {'n_estimators' : [100, 200,300],
#               'learning_rate':[0.01,0.1,0.2,0.3]
#              }

# best_param = grid_search_all_model(X_train,y_train,model,params_xgb)

# # for i in np.arange(12):
# #     model_list_xgb.append(XGBRegressor(learning_rate = 0.1,
# #                      n_jobs = 6,
# #                      n_estimators = 200,
# #                      eval_metric = 'rmse',
# #                      random_state = seed))

# end = time.time()
# print('Time:',end - start)


In [ ]:
start = time.time()

model_list_xgb = []
for i in np.arange(12):
    model_list_xgb.append(XGBRegressor(learning_rate = 0.1,
                                        n_jobs = 6,
                                        n_estimators = 200,
                                        random_state = seed))



val_error,tr_error = cross_validation(X_train,y_train,model_list_xgb,psnr,folds= 3)
print(' Validation error:',val_error,'\n','Train error:',tr_error,'\n\n')

end = time.time()
print('Time:',end - start)

In [101]:
start = time.time()

val_error,tr_error = cross_validation(X_train,y_train,model_list_xgb,psnr,folds= 3)
print(' Validation error:',val_error,'\n','Train error:',tr_error,'\n\n')

end = time.time()
print('Time:',end - start)

 Validation error: [24.95, 25.12, 25.07] 
 Train error: [25.55, 25.46, 25.48] 


Time: 166.49743270874023


In [105]:
start = time.time()

prediction = Prediction(X_train,y_train,X_test,model_list_xgb,test = True)
print('PSNR:',psnr(prediction, y_test),'\n\n')

end = time.time()
print('Time:',end - start)

PSNR: 25.053560815915965 


Time: 46.624295711517334


In [ ]:
# def get_X_and_y(LR,HR):
    
#     def get_neighbor_X(a,i,j):
#         return([a[i-1,j-1],a[i-1,j],a[i-1,j+1],a[i,j-1],a[i,j+1],a[i+1,j-1],a[i+1,j],a[i+1,j+1]],a[i,j])

#     def get_neighbor_y(a,i,j):
#         return([a[i,j],a[i+1,j],a[i,j+1],a[i+1,j+1]],a[i,j])
    
#     BLACK = [0, 0, 0]
#     image_padding = cv2.copyMakeBorder(LR, 1 , 1, 1, 1, cv2.BORDER_CONSTANT, value=BLACK)
    
#     height = LR.shape[1]
#     width = LR.shape[0]
    
#     y1 = np.zeros((height*width,4))
#     y2 = np.zeros((height*width,4))
#     y3 = np.zeros((height*width,4))
    
#     X1 = np.zeros((height*width,8))
#     X2 = np.zeros((height*width,8))
#     X3 = np.zeros((height*width,8))
    
#     result = [X1,X2,X3]
#     Y = [y1,y2,y3]
    
#     for X,y,channel in zip(result,Y,channels):
#         index = 0
#         for i,iy in zip(np.arange(1,width + 1),np.arange(0,width_HR,2)):
#             for j,jy in zip(np.arange(1,height + 1),np.arange(0,height_HR,2)):
#                 neighbor,central = get_neighbor_X(image_padding[:,:,channel],i,j)
#                 y_neigbor,y_central = get_neighbor_y(HR[:,:,channel],iy,jy)
#                 # Get X
#                 X[index] = neighbor - central
#                 # Get y
#                 y[index] = y_neigbor - y_central
#                 index +=1
    
    
#     # Stack X&Y to 3d
#     return np.dstack(result),np.dstack(Y)